<a href="https://colab.research.google.com/github/kronosjujo/optimizacion/blob/main/MODELO_MATEM%C3%81TICO_DE_OPTIMIZACI%C3%93N.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalación del paquete pulp (Biblioteca de Python utilizada para resolver problemas de optimización lineal)**

In [1]:
# Alternativa 1: Instalación con pip3
!pip3 install pulp

# Alternativa 2: Instalación forzada
!pip install --force-reinstall pulp

#Verificación que se instaló correctamente
!pip show pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 59.2 MB/s eta 0:00:00
  Using cached pulp-3.2.1-py3-none-any.whl.metadata (6.9 kB)
Using cached pulp-3.2.1-py3-none-any.whl (16.4 MB)
  Attempting uninstall: pulp
    Found existing installation: PuLP 3.2.1
    Uninstalling PuLP-3.2.1:
      Successfully uninstalled PuLP-3.2.1
Name: PuLP
Version: 3.2.1
Summary: PuLP is an LP modeler written in python. PuLP can generate MPS or LP files and call GLPK, COIN CLP/CBC, CPLEX, and GUROBI to solve linear problems.
Home-page: 
Author: J.S. Roy
Author-email: "S.A. Mitchell" <pulp@stuartmitchell.com>, Franco Peschiera <pchtsp@gmail.com>
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: 



## IMPLEMENTACIÓN DE UN MODELO MATEMÁTICO DE OPTIMIZACIÓN PARA LA ASIGNACIÓN DE HORARIOS DE CLASE Y RECURSOS TECNOLÓGICOS MEDIANTE PYTHON EN UN INSTITUTO DE LENGUAS EXTRANJERAS, PERIODO 2024-6B

# **ETAPA 1: ASIGNACIÓN DE PROFESORES A LAS CLASES**

In [3]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, LpStatus, PULP_CBC_CMD
import sys
from typing import Dict, List, Tuple

class AsignacionProfesores:
    """
    Clase para resolver el problema de asignación de profesores a clases
    usando programación lineal entera.
    """

    def __init__(self):
        self.df_clases = None
        self.df_preferencias = None
        self.profesores = []
        self.clases_idx = []
        self.planta_profes = []
        self.contrato_profes = []
        self.modulos = []
        self.horarios = []
        self.clase_info = {}
        self.preferencia_combinada = {}
        self.max_clases = {}

        # Nombres de columnas configurables
        self.COLUMNA_MODULO = 'MÓDULO'
        self.COLUMNA_HORARIO = 'HORARIO'
        self.COLUMNA_PARALELO = 'PARALELO'

    def cargar_datos(self, ruta_clases: str = "/content/clases.xlsx",
                     ruta_preferencias: str = "/content/preferencias_profesores.xlsx"):
        """
        Carga los datos desde archivos Excel y valida la estructura.

        Args:
            ruta_clases: Ruta del archivo de clases
            ruta_preferencias: Ruta del archivo de preferencias
        """
        print("--- CARGA DE DATOS ---")
        try:
            print("Cargando datos desde archivos Excel...")
            self.df_clases = pd.read_excel(ruta_clases)
            self.df_preferencias = pd.read_excel(ruta_preferencias, index_col="PROFESOR")
            print("✓ Datos cargados correctamente.")

            self._validar_estructura_datos()

        except FileNotFoundError as e:
            print(f"❌ Error: Archivo no encontrado - {e}")
            sys.exit()
        except Exception as e:
            print(f"❌ Error al cargar archivos: {e}")
            sys.exit()

    def _validar_estructura_datos(self):
        """Valida que las columnas requeridas existan en los DataFrames."""
        print("Validando estructura de datos...")

        # Validar columnas en df_clases
        columnas_requeridas_clases = [self.COLUMNA_MODULO, self.COLUMNA_HORARIO]
        for col in columnas_requeridas_clases:
            if col not in self.df_clases.columns:
                raise ValueError(f"Columna '{col}' no encontrada en clases.xlsx")

        # Crear columna PARALELO si no existe
        if self.COLUMNA_PARALELO not in self.df_clases.columns:
            print(f"⚠️ Columna '{self.COLUMNA_PARALELO}' no encontrada. Usando 'N/A'.")
            self.df_clases[self.COLUMNA_PARALELO] = 'N/A'

        # Convertir horarios a string
        if not pd.api.types.is_string_dtype(self.df_clases[self.COLUMNA_HORARIO]):
            print(f"⚠️ Convirtiendo columna '{self.COLUMNA_HORARIO}' a texto...")
            self.df_clases[self.COLUMNA_HORARIO] = self.df_clases[self.COLUMNA_HORARIO].astype(str)

        print("✓ Estructura de datos validada.")

    def preparar_conjuntos_parametros(self):
        """
        Identifica y prepara todos los conjuntos y parámetros del modelo.
        """
        print("\n--- PREPARACIÓN DE CONJUNTOS Y PARÁMETROS ---")

        # Identificar profesores y tipos
        self.profesores = self.df_preferencias.index.tolist()
        self.clases_idx = self.df_clases.index.tolist()
        self.planta_profes = [p for p in self.profesores if p.startswith("Planta")]
        self.contrato_profes = [p for p in self.profesores if not p.startswith("Planta")]

        print(f"📊 Total profesores: {len(self.profesores)} (Planta: {len(self.planta_profes)}, Contrato: {len(self.contrato_profes)})")
        print(f"📚 Total clases: {len(self.clases_idx)}")

        # Identificar módulos y horarios
        self._identificar_modulos_horarios()

        # Preparar información de clases
        self.clase_info = self.df_clases.set_index(self.df_clases.index).to_dict('index')

        # Calcular preferencias
        self._calcular_preferencias()

        print("✓ Conjuntos y parámetros preparados.")

    def _identificar_modulos_horarios(self):
        """Identifica las columnas de módulos y horarios en el DataFrame de preferencias."""
        try:
            all_columns = self.df_preferencias.columns.tolist()
            col_clases = all_columns[0]  # Primera columna: max clases

            # Encontrar índices de referencia
            idx_col_clases = all_columns.index(col_clases)
            idx_last_module = all_columns.index('A4')

            # Extraer módulos y horarios
            self.modulos = all_columns[idx_col_clases + 1 : idx_last_module + 1]
            self.horarios = all_columns[idx_last_module + 1 :]

            print(f"📋 Módulos identificados: {self.modulos}")
            print(f"🕐 Horarios identificados: {self.horarios}")

            if not self.modulos or not self.horarios:
                raise ValueError("No se pudieron identificar módulos o horarios")

        except (ValueError, IndexError) as e:
            print(f"❌ Error identificando columnas: {e}")
            sys.exit()

    def _calcular_preferencias(self):
        """Calcula las preferencias combinadas (módulo × horario) para cada profesor-clase."""
        print("Calculando preferencias combinadas...")

        # Extraer preferencias por módulo y horario
        pref_mod = self.df_preferencias[self.modulos].stack().to_dict()
        pref_hor = self.df_preferencias[self.horarios].stack().to_dict()

        # Obtener máximo de clases por profesor
        col_clases = self.df_preferencias.columns[0]
        self.max_clases = self.df_preferencias[col_clases].to_dict()

        # Calcular preferencias combinadas
        coincidencias = 0
        for profesor in self.profesores:
            for clase_idx in self.clases_idx:
                info_clase = self.clase_info[clase_idx]
                modulo_clase = info_clase[self.COLUMNA_MODULO]
                horario_clase = info_clase[self.COLUMNA_HORARIO].strip()

                pref_modulo = int(pref_mod.get((profesor, modulo_clase), 0))
                pref_horario = int(pref_hor.get((profesor, horario_clase), 0))

                # Preferencia combinada: 1 si ambas son 1, 0 en caso contrario
                pref_combinada = pref_modulo * pref_horario
                self.preferencia_combinada[profesor, clase_idx] = pref_combinada

                if pref_combinada == 1:
                    coincidencias += 1

        print(f"✓ {coincidencias} coincidencias perfectas (preferencia = 1) encontradas.")

    def construir_modelo(self):
        """
        Construye el modelo de programación lineal con todas las restricciones.
        """
        print("\n--- CONSTRUCCIÓN DEL MODELO ---")

        # Crear problema de maximización
        self.prob = LpProblem("Asignacion_Profesores_E1", LpMaximize)

        # Variables de decisión: x[profesor, clase] ∈ {0, 1}
        self.x = LpVariable.dicts(
            "Asignacion",
            [(p, c) for p in self.profesores for c in self.clases_idx],
            cat='Binary'
        )

        # Función objetivo: Maximizar suma de preferencias
        self.prob += lpSum(
            self.preferencia_combinada[p, c] * self.x[p, c]
            for p in self.profesores for c in self.clases_idx
        ), "Maximizar_Preferencias_Totales"

        print("✓ Función objetivo definida.")

        # Añadir restricciones
        self._añadir_restricciones()

        print("✓ Modelo construido completamente.")

    def _añadir_restricciones(self):
        """Añade todas las restricciones al modelo."""
        print("Añadiendo restricciones...")

        # R1: Cobertura total - Cada clase debe ser asignada a exactamente un profesor
        for c in self.clases_idx:
            self.prob += lpSum(self.x[p, c] for p in self.profesores) == 1, f"Cobertura_Clase_{c}"

        # R2: Carga obligatoria de planta - Profesores de planta deben tener exactamente 3 clases
        for p in self.planta_profes:
            self.prob += lpSum(self.x[p, c] for c in self.clases_idx) == 3, f"Carga_Planta_{p}"

        # R3: Carga máxima de contrato - Profesores de contrato no pueden exceder su máximo
        for p in self.contrato_profes:
            self.prob += lpSum(self.x[p, c] for c in self.clases_idx) <= self.max_clases[p], f"Carga_Max_Contrato_{p}"

        # R4: Conflicto de horarios - Un profesor no puede estar en dos clases al mismo tiempo
        for profesor in self.profesores:
            for horario in self.horarios:
                clases_en_horario = [
                    c for c in self.clases_idx
                    if self.clase_info[c][self.COLUMNA_HORARIO].strip() == horario
                ]
                if clases_en_horario:
                    nombre_horario_seguro = "".join(filter(str.isalnum, horario))
                    self.prob += lpSum(self.x[profesor, c] for c in clases_en_horario) <= 1, \
                                f"Sin_Conflicto_{profesor}_{nombre_horario_seguro}"

        # R5: Módulos permitidos - Solo asignar profesores a módulos que pueden enseñar
        pref_mod = self.df_preferencias[self.modulos].stack().to_dict()
        for profesor in self.profesores:
            for clase in self.clases_idx:
                modulo_clase = self.clase_info[clase][self.COLUMNA_MODULO]
                puede_ensenar = pref_mod.get((profesor, modulo_clase), 0)
                self.prob += self.x[profesor, clase] <= puede_ensenar, \
                            f"Modulo_Permitido_{profesor}_{clase}"

        print("✓ Todas las restricciones añadidas.")

    def resolver_modelo(self):
        """
        Resuelve el modelo de optimización.
        """
        print("\n--- RESOLUCIÓN ---")
        print("Resolviendo modelo de optimización...")

        solver = PULP_CBC_CMD(msg=0)  # Solver silencioso
        self.prob.solve(solver)

        print(f"✓ Modelo resuelto. Estado: {LpStatus[self.prob.status]}")
        return LpStatus[self.prob.status]

    def generar_resultados(self, exportar_excel: bool = True):
        """
        Genera y presenta los resultados de la asignación.

        Args:
            exportar_excel: Si exportar resultados a Excel
        """
        print("\n--- RESULTADOS ---")

        estado = LpStatus[self.prob.status]
        print(f"📊 Estado de la solución: {estado}")

        # Mostrar valor objetivo
        if self.prob.objective is not None:
            valor_objetivo = self.prob.objective.value()
            print(f"🎯 Valor óptimo (suma de preferencias): {valor_objetivo or 'N/A'}")

        if estado == 'Optimal':
            self._procesar_solucion_optima(exportar_excel)
        elif estado == 'Infeasible':
            self._manejar_infactibilidad()
        else:
            print(f"❌ No se encontró solución óptima. Estado: {estado}")

    def _procesar_solucion_optima(self, exportar_excel: bool):
        """Procesa y presenta la solución óptima encontrada."""
        print("\n🎉 ¡SOLUCIÓN ÓPTIMA ENCONTRADA!")

        # Extraer asignaciones
        asignaciones = []
        carga_profesores = {p: 0 for p in self.profesores}

        for (profesor, clase), variable in self.x.items():
            if variable.varValue > 0.9:  # Asignado
                info_clase = self.clase_info[clase]
                preferencia = self.preferencia_combinada.get((profesor, clase), 0)

                asignaciones.append({
                    "Profesor": profesor,
                    "Clase_ID": clase,
                    "Modulo": info_clase[self.COLUMNA_MODULO],
                    "Horario": info_clase[self.COLUMNA_HORARIO].strip(),
                    "Paralelo": info_clase.get(self.COLUMNA_PARALELO, 'N/A'),
                    "Preferencia_Cumplida": preferencia
                })
                carga_profesores[profesor] += 1

        # Crear DataFrame de resultados
        df_resultados = pd.DataFrame(asignaciones)
        df_resultados.sort_values(by=["Profesor", "Horario"], inplace=True)

        # Mostrar asignaciones
        print("\n📋 ASIGNACIONES DETALLADAS:")
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1200):
            print(df_resultados)

        # Exportar a Excel si se solicita
        if exportar_excel:
            self._exportar_excel(df_resultados)

        # Mostrar resumen de carga
        self._mostrar_resumen_carga(carga_profesores)

        # Verificar profesores sin asignación
        self._verificar_profesores_sin_asignacion(carga_profesores)

    def _exportar_excel(self, df_resultados: pd.DataFrame):
        """Exporta los resultados a un archivo Excel."""
        try:
            nombre_archivo = "asignacion_profesores_E1.xlsx"
            df_resultados.to_excel(nombre_archivo, index=False)
            print(f"\n💾 ¡Resultados exportados a '{nombre_archivo}'!")
        except Exception as e:
            print(f"\n❌ Error al exportar: {e}")

    def _mostrar_resumen_carga(self, carga_profesores: Dict[str, int]):
        """Muestra el resumen de carga por profesor."""
        print("\n📊 RESUMEN DE CARGA POR PROFESOR:")
        print("-" * 60)

        for profesor in self.profesores:
            tipo = "Planta" if profesor in self.planta_profes else "Contrato"
            maximo = self.max_clases[profesor]
            asignadas = carga_profesores[profesor]

            # Indicador visual de cumplimiento
            if tipo == "Planta":
                indicador = "✓" if asignadas == 3 else "❌"
            else:
                indicador = "✓" if asignadas > 0 else "⚠️"

            print(f"{indicador} {profesor:12} ({tipo:8}): {asignadas:2d} clases asignadas (Máx: {maximo})")

    def _verificar_profesores_sin_asignacion(self, carga_profesores: Dict[str, int]):
        """Verifica y reporta profesores de contrato sin asignaciones."""
        sin_asignacion = [p for p in self.contrato_profes if carga_profesores[p] == 0]

        print("\n🔍 VERIFICACIÓN DE COBERTURA:")
        if not sin_asignacion:
            print("✅ ¡EXCELENTE! Todos los profesores de contrato tienen al menos una clase.")
        else:
            print("⚠️  ADVERTENCIA: Profesores de contrato sin clases asignadas:")
            for profesor in sin_asignacion:
                print(f"   - {profesor}")
            print("   💡 Posibles causas: incompatibilidad módulo/horario o alta competencia.")

    def _manejar_infactibilidad(self):
        """Maneja el caso de problema infactible."""
        print("\n❌ PROBLEMA INFACTIBLE")
        print("No existe solución que cumpla todas las restricciones.")
        print("\n💡 Posibles causas:")
        print("   - Insuficientes profesores para cubrir todas las clases")
        print("   - Restricciones de horario muy estrictas")
        print("   - Incompatibilidad entre módulos y profesores disponibles")
        print("   - Carga obligatoria de planta demasiado alta")

    def ejecutar_proceso_completo(self, ruta_clases: str = "/content/clases.xlsx",
                                  ruta_preferencias: str = "/content/preferencias_profesores.xlsx"):
        """
        Ejecuta todo el proceso de asignación de principio a fin.

        Args:
            ruta_clases: Ruta del archivo de clases
            ruta_preferencias: Ruta del archivo de preferencias

        Returns:
            str: Estado final de la solución
        """
        print("=" * 80)
        print("🎓 SISTEMA DE ASIGNACIÓN DE PROFESORES - ETAPA 1")
        print("=" * 80)

        try:
            # Ejecutar todas las etapas
            self.cargar_datos(ruta_clases, ruta_preferencias)
            self.preparar_conjuntos_parametros()
            self.construir_modelo()
            estado = self.resolver_modelo()
            self.generar_resultados()

            print("\n" + "=" * 80)
            print("✅ PROCESO COMPLETADO EXITOSAMENTE")
            print("=" * 80)

            return estado

        except Exception as e:
            print(f"\n❌ ERROR CRÍTICO: {e}")
            print("=" * 80)
            return "Error"


# ===== FUNCIÓN PRINCIPAL =====
def main():
    """Función principal para ejecutar el sistema de asignación."""
    asignador = AsignacionProfesores()
    estado_final = asignador.ejecutar_proceso_completo()
    return estado_final


# ===== EJECUCIÓN =====
if __name__ == "__main__":
    main()

🎓 SISTEMA DE ASIGNACIÓN DE PROFESORES - ETAPA 1
--- CARGA DE DATOS ---
Cargando datos desde archivos Excel...
✓ Datos cargados correctamente.
Validando estructura de datos...
✓ Estructura de datos validada.

--- PREPARACIÓN DE CONJUNTOS Y PARÁMETROS ---
📊 Total profesores: 30 (Planta: 2, Contrato: 28)
📚 Total clases: 46
📋 Módulos identificados: ['B1', 'B2', 'B3', 'B4', 'B5', 'I1', 'I2', 'I3', 'I4', 'A1', 'A2', 'A3', 'A4']
🕐 Horarios identificados: ['08H00-10H00', '15H00-17H00', '17H00-19H00', '19H00-21H00']
Calculando preferencias combinadas...
✓ 831 coincidencias perfectas (preferencia = 1) encontradas.
✓ Conjuntos y parámetros preparados.

--- CONSTRUCCIÓN DEL MODELO ---
✓ Función objetivo definida.
Añadiendo restricciones...
✓ Todas las restricciones añadidas.
✓ Modelo construido completamente.

--- RESOLUCIÓN ---
Resolviendo modelo de optimización...
✓ Modelo resuelto. Estado: Optimal

--- RESULTADOS ---
📊 Estado de la solución: Optimal
🎯 Valor óptimo (suma de preferencias): 46.0



# **ETAPA 2: ASIGNACIÓN DE CUENTAS DE VIDEOCONFERENCIA**



In [4]:
import pandas as pd
import pulp
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus, PULP_CBC_CMD
import sys
from collections import defaultdict
import itertools
import time
import warnings

# Configuración para evitar warnings innecesarios
warnings.filterwarnings('ignore')

def cargar_datos_etapa1(archivo_path="/content/asignacion_profesores_E1.xlsx"):
    """
    Carga los datos de la salida de la Etapa 1.

    Args:
        archivo_path (str): Ruta del archivo Excel de la Etapa 1

    Returns:
        pd.DataFrame: DataFrame con los datos cargados
    """
    try:
        print("📂 Cargando datos desde la salida de la Etapa 1...")
        df_etapa1 = pd.read_excel(archivo_path)
        print(f"✅ Datos cargados correctamente. Registros encontrados: {len(df_etapa1)}")
        return df_etapa1
    except FileNotFoundError:
        print("❌ Error: No se encontró el archivo 'asignacion_profesores_E1.xlsx'.")
        print("   Por favor, ejecuta la Etapa 1 primero.")
        sys.exit()
    except Exception as e:
        print(f"❌ Error al cargar el archivo Excel de la Etapa 1: {e}")
        sys.exit()

def procesar_horarios(df_etapa1):
    """
    Procesa los horarios de los profesores y crea estructuras de datos necesarias.

    Args:
        df_etapa1 (pd.DataFrame): DataFrame con datos de la Etapa 1

    Returns:
        tuple: (horarios_por_profesor, conjunto_horarios_unicos)
    """
    horarios_por_profesor = defaultdict(set)
    all_unique_horarios = set()

    for index, row in df_etapa1.iterrows():
        # Validar que la columna Horario existe y no es nula
        if 'Horario' not in row or pd.isna(row['Horario']):
            print(f"⚠️  Advertencia: Fila {index} no tiene Horario o es nulo. Omitiendo.")
            continue

        horario_limpio = str(row['Horario']).strip()
        profesor = row['Profesor']

        horarios_por_profesor[profesor].add(horario_limpio)
        all_unique_horarios.add(horario_limpio)

    return horarios_por_profesor, list(all_unique_horarios)

def calcular_pares_contiguos(profesores_activos, horarios_por_profesor):
    """
    Calcula pares de profesores con horarios contiguos.

    Args:
        profesores_activos (list): Lista de profesores activos
        horarios_por_profesor (dict): Diccionario de horarios por profesor

    Returns:
        set: Conjunto de pares contiguos
    """
    # Orden canónico de horarios para determinar contigüidad
    orden_horarios = ['08H00-10H00', '15H00-17H00', '17H00-19H00', '19H00-21H00']
    horario_a_indice = {h: i for i, h in enumerate(orden_horarios)}

    def check_contiguity(h1, h2):
        """Verifica si dos horarios son contiguos"""
        idx1 = horario_a_indice.get(h1)
        idx2 = horario_a_indice.get(h2)
        if idx1 is None or idx2 is None:
            return False  # Horario no estándar
        return abs(idx1 - idx2) == 1

    pares_contiguos = set()

    for p1, p2 in itertools.combinations(profesores_activos, 2):
        horarios_p1 = horarios_por_profesor.get(p1, set())
        horarios_p2 = horarios_por_profesor.get(p2, set())

        # Verificar si existe algún par de horarios contiguos
        tiene_contiguo = any(
            check_contiguity(h1, h2)
            for h1 in horarios_p1
            for h2 in horarios_p2
        )

        if tiene_contiguo:
            pares_contiguos.add(tuple(sorted((p1, p2))))

    return pares_contiguos

def calcular_pares_solapamiento(profesores_activos, horarios_por_profesor):
    """
    Calcula pares de profesores con solapamiento de horarios.

    Args:
        profesores_activos (list): Lista de profesores activos
        horarios_por_profesor (dict): Diccionario de horarios por profesor

    Returns:
        tuple: (pares_solapamiento_por_horario, total_pares_solapamiento)
    """
    # Crear mapeo de horario -> lista de profesores
    profesores_por_horario = defaultdict(list)
    for profesor in profesores_activos:
        for horario in horarios_por_profesor.get(profesor, set()):
            profesores_por_horario[horario].append(profesor)

    # Encontrar pares que comparten horarios
    pares_solapamiento_por_horario = defaultdict(set)

    for horario, lista_profesores in profesores_por_horario.items():
        if len(lista_profesores) >= 2:
            for p1, p2 in itertools.combinations(lista_profesores, 2):
                pares_solapamiento_por_horario[horario].add(tuple(sorted((p1, p2))))

    total_pares = sum(len(pares) for pares in pares_solapamiento_por_horario.values())

    return pares_solapamiento_por_horario, total_pares

def crear_modelo_optimizacion(profesores_activos, cuentas_disponibles,
                            pares_contiguos, pares_solapamiento_por_horario,
                            num_clases_por_profesor, config):
    """
    Crea el modelo de optimización para asignación de cuentas VC.

    Args:
        profesores_activos (list): Lista de profesores activos
        cuentas_disponibles (list): Lista de cuentas disponibles
        pares_contiguos (set): Pares de profesores con horarios contiguos
        pares_solapamiento_por_horario (dict): Pares con solapamiento por horario
        num_clases_por_profesor (dict): Número de clases por profesor
        config (dict): Configuración del modelo

    Returns:
        tuple: (problema, variables_y, variables_u, variables_v)
    """
    print("🔧 Definiendo el modelo de optimización...")

    # Crear problema de minimización
    prob = LpProblem("Asignacion_Cuentas_VC_Optimizada", LpMinimize)

    # Variables de decisión
    # y[p,a] = 1 si profesor p es asignado a cuenta a
    y = LpVariable.dicts("AsignacionCuenta",
                        [(p, a) for p in profesores_activos for a in cuentas_disponibles],
                        cat='Binary')

    # u[a] = 1 si cuenta adicional a es utilizada
    u = LpVariable.dicts("UsoCuentaAdicional", config['cuentas_adicionales'], cat='Binary')

    # v[a] = 1 si cuenta base a es utilizada
    v = LpVariable.dicts("UsoCuentaBase", config['cuentas_base'], cat='Binary')

    # Función objetivo: minimizar costo ponderado de cuentas
    prob += (config['peso_adicional'] * lpSum(u[a] for a in config['cuentas_adicionales']) +
             config['peso_base'] * lpSum(v[a] for a in config['cuentas_base'])), \
             "Minimizar_Costo_Ponderado_Cuentas"

    # RESTRICCIONES
    print("📋 Añadiendo restricciones al modelo...")

    # R1: Cada profesor debe ser asignado a exactamente una cuenta
    for p in profesores_activos:
        prob += lpSum(y[p, a] for a in cuentas_disponibles) == 1, \
                f"AsignacionUnica_{p}"

    # R2: Máximo 3 profesores por cuenta
    for a in cuentas_disponibles:
        prob += lpSum(y[p, a] for p in profesores_activos) <= 3, \
                f"LimiteCompartir_{a}"

    # R3: Exclusividad para profesores con 3 clases
    profesores_3_clases = [p for p in profesores_activos
                          if num_clases_por_profesor.get(p, 0) == 3]

    if profesores_3_clases:
        print(f"   👤 Aplicando exclusividad para {len(profesores_3_clases)} profesores con 3 clases")
        for p in profesores_3_clases:
            for a in cuentas_disponibles:
                otros_profesores = [p_prime for p_prime in profesores_activos if p_prime != p]
                prob += lpSum(y[p_prime, a] for p_prime in otros_profesores) <= 3 * (1 - y[p, a]), \
                        f"Exclusividad3Clases_{p}_{a}"

    # R4: Prohibir compartir cuenta entre profesores con horarios contiguos
    if pares_contiguos:
        print(f"   ⏰ Aplicando restricción de no contigüidad para {len(pares_contiguos)} pares")
        for p1, p2 in pares_contiguos:
            if p1 in profesores_activos and p2 in profesores_activos:
                for a in cuentas_disponibles:
                    prob += y[p1, a] + y[p2, a] <= 1, \
                            f"NoContiguo_{p1}_{p2}_{a}"

    # R5: Prohibir conflictos de horario en la misma cuenta
    total_pares_solapamiento = sum(len(pares) for pares in pares_solapamiento_por_horario.values())
    if total_pares_solapamiento > 0:
        print(f"   🔄 Aplicando restricción de no solapamiento para {total_pares_solapamiento} pares")
        for horario, pares_en_horario in pares_solapamiento_por_horario.items():
            horario_seguro = "".join(filter(str.isalnum, horario))
            for p1, p2 in pares_en_horario:
                if p1 in profesores_activos and p2 in profesores_activos:
                    for a in cuentas_disponibles:
                        prob += y[p1, a] + y[p2, a] <= 1, \
                                f"NoSolapamiento_{p1}_{p2}_{horario_seguro}_{a}"

    # R6: Activación de cuentas adicionales
    for a in config['cuentas_adicionales']:
        for p in profesores_activos:
            prob += y[p, a] <= u[a], f"ActivarUsoAdicional_{p}_{a}"

    # R7: Activación de cuentas base
    for a in config['cuentas_base']:
        for p in profesores_activos:
            prob += y[p, a] <= v[a], f"ActivarUsoBase_{p}_{a}"

    print("✅ Modelo definido correctamente")
    return prob, y, u, v

def resolver_problema(prob, tiempo_limite=600):
    """
    Resuelve el problema de optimización.

    Args:
        prob: Problema de PuLP
        tiempo_limite (int): Tiempo límite en segundos

    Returns:
        int: Estado de la solución
    """
    print(f"🚀 Resolviendo el problema... (Tiempo límite: {tiempo_limite} segundos)")

    solver = PULP_CBC_CMD(timeLimit=tiempo_limite, gapRel=0.01, msg=True)
    prob.solve(solver)

    return prob.status

def procesar_resultados(prob, y, u, v, profesores_activos, cuentas_disponibles,
                       config, num_clases_por_profesor, horarios_por_profesor):
    """
    Procesa y presenta los resultados de la optimización.

    Args:
        prob: Problema resuelto
        y, u, v: Variables del modelo
        profesores_activos (list): Lista de profesores
        cuentas_disponibles (list): Lista de cuentas
        config (dict): Configuración
        num_clases_por_profesor (dict): Número de clases por profesor
        horarios_por_profesor (dict): Horarios por profesor

    Returns:
        pd.DataFrame: DataFrame con los resultados
    """
    status_text = LpStatus[prob.status]
    print(f"\n📊 Estado de la solución: {status_text}")

    if pulp.LpStatus[prob.status] != "Optimal":
        if pulp.LpStatus[prob.status] == "Infeasible":
            print("❌ El problema es INFACTIBLE. Revisar las restricciones.")
        else:
            print(f"⚠️ Estado del problema: {pulp.LpStatus[prob.status]}")
        return None

    # Calcular estadísticas de uso de cuentas
    try:
        num_adicionales_usadas = int(sum(u[a].varValue or 0 for a in config['cuentas_adicionales']))
        num_base_usadas = int(sum(v[a].varValue or 0 for a in config['cuentas_base']))
        num_total_cuentas = num_base_usadas + num_adicionales_usadas

        print(f"\n📈 RESUMEN DE CUENTAS:")
        print(f"   • Cuentas base utilizadas: {num_base_usadas}/{config['num_base']}")
        print(f"   • Cuentas adicionales necesarias: {num_adicionales_usadas}")
        print(f"   • TOTAL de cuentas utilizadas: {num_total_cuentas}")

        cuentas_base_libres = config['num_base'] - num_base_usadas
        if cuentas_base_libres > 0:
            print(f"   • Cuentas base disponibles sin usar: {cuentas_base_libres}")

    except (TypeError, AttributeError):
        print("⚠️  No se pudieron calcular las estadísticas de cuentas")

    # Procesar asignaciones individuales
    resultados = []
    cuentas_usadas = defaultdict(list)

    for p in profesores_activos:
        cuenta_asignada = None

        for a in cuentas_disponibles:
            if hasattr(y[p, a], 'varValue') and y[p, a].varValue and y[p, a].varValue > 0.9:
                cuenta_asignada = a
                cuentas_usadas[a].append(p)
                break

        if cuenta_asignada is None:
            print(f"⚠️  No se encontró asignación para el profesor: {p}")
            cuenta_asignada = "ERROR"

        resultados.append({
            "Profesor": p,
            "NumClases": num_clases_por_profesor.get(p, 0),
            "CuentaVC_Asignada": cuenta_asignada,
            "Horarios": ", ".join(sorted(horarios_por_profesor.get(p, set())))
        })

    # Crear DataFrame de resultados
    df_resultados = pd.DataFrame(resultados)

    # Ordenar resultados
    try:
        if "ERROR" not in df_resultados["CuentaVC_Asignada"].values:
            df_resultados = df_resultados.astype({"CuentaVC_Asignada": int})
            df_resultados.sort_values(by=["CuentaVC_Asignada", "Profesor"], inplace=True)
    except (ValueError, TypeError):
        df_resultados.sort_values(by=["Profesor"], inplace=True)

    # Mostrar resultados
    print(f"\n👥 ASIGNACIÓN DE CUENTAS POR PROFESOR:")
    print(df_resultados.to_string(index=False))

    print(f"\n🏢 RESUMEN DE USO POR CUENTA:")
    for cuenta in sorted(cuentas_usadas.keys()):
        profesores_en_cuenta = cuentas_usadas[cuenta]
        tipo_cuenta = "Base" if cuenta <= config['num_base'] else "Adicional"
        print(f"   Cuenta {cuenta} ({tipo_cuenta}): {len(profesores_en_cuenta)} profesor(es)")
        print(f"      → {', '.join(sorted(profesores_en_cuenta))}")

    return df_resultados

def exportar_resultados(df_resultados, archivo_salida="asignacion__VC_E2.xlsx"):
    """
    Exporta los resultados a un archivo Excel.

    Args:
        df_resultados (pd.DataFrame): DataFrame con los resultados
        archivo_salida (str): Nombre del archivo de salida
    """
    try:
        # Exportar solo las columnas necesarias para la Etapa 3
        df_export = df_resultados[['Profesor', 'CuentaVC_Asignada']].copy()
        df_export.to_excel(archivo_salida, index=False)
        print(f"\n💾 Resultados exportados exitosamente a '{archivo_salida}'")
    except Exception as e:
        print(f"❌ Error al exportar resultados: {e}")

def main():
    """Función principal que ejecuta todo el proceso de la Etapa 2."""
    print("🎯 === INICIANDO ETAPA 2: ASIGNACIÓN DE CUENTAS VC ===")
    start_time = time.time()

    # 1. Cargar datos de la Etapa 1
    df_etapa1 = cargar_datos_etapa1()

    # 2. Procesar datos básicos
    profesores_activos = df_etapa1['Profesor'].unique().tolist()
    if not profesores_activos:
        print("❌ No hay profesores activos. Terminando proceso.")
        sys.exit()

    num_clases_por_profesor = df_etapa1['Profesor'].value_counts().to_dict()
    horarios_por_profesor, horarios_unicos = procesar_horarios(df_etapa1)

    # 3. Configuración del modelo
    config = {
        'num_base': 20,  # Número de cuentas base disponibles
        'num_max': len(profesores_activos),  # Cota superior teórica
        'peso_adicional': 1000,  # Peso para cuentas adicionales
        'peso_base': 1  # Peso para cuentas base
    }

    config['cuentas_base'] = list(range(1, config['num_base'] + 1))
    config['cuentas_adicionales'] = list(range(config['num_base'] + 1, config['num_max'] + 1)) \
                                   if config['num_max'] > config['num_base'] else []
    cuentas_disponibles = config['cuentas_base'] + config['cuentas_adicionales']

    # 4. Calcular restricciones
    print("\n🔍 Analizando restricciones...")
    pares_contiguos = calcular_pares_contiguos(profesores_activos, horarios_por_profesor)
    pares_solapamiento, total_solapamiento = calcular_pares_solapamiento(profesores_activos, horarios_por_profesor)

    print(f"   📊 Estadísticas del problema:")
    print(f"      • Profesores activos: {len(profesores_activos)}")
    print(f"      • Pares con horarios contiguos: {len(pares_contiguos)}")
    print(f"      • Pares con solapamiento de horario: {total_solapamiento}")
    print(f"      • Cuentas disponibles: {len(cuentas_disponibles)} (Base: {len(config['cuentas_base'])}, Adicionales: {len(config['cuentas_adicionales'])})")

    # 5. Crear y resolver modelo
    prob, y, u, v = crear_modelo_optimizacion(
        profesores_activos, cuentas_disponibles, pares_contiguos,
        pares_solapamiento, num_clases_por_profesor, config
    )

    status = resolver_problema(prob, tiempo_limite=600)

    # 6. Procesar y mostrar resultados
    df_resultados = procesar_resultados(
        prob, y, u, v, profesores_activos, cuentas_disponibles,
        config, num_clases_por_profesor, horarios_por_profesor
    )

    # 7. Exportar resultados si son válidos
    if df_resultados is not None and "ERROR" not in df_resultados["CuentaVC_Asignada"].values:
        exportar_resultados(df_resultados)

    # 8. Tiempo total de ejecución
    end_time = time.time()
    print(f"\n⏱️  Tiempo total de ejecución: {end_time - start_time:.2f} segundos")
    print("🏁 === FIN DE LA ETAPA 2 ===")

# Ejecutar el programa principal
if __name__ == "__main__":
    main()

🎯 === INICIANDO ETAPA 2: ASIGNACIÓN DE CUENTAS VC ===
📂 Cargando datos desde la salida de la Etapa 1...
✅ Datos cargados correctamente. Registros encontrados: 46

🔍 Analizando restricciones...
   📊 Estadísticas del problema:
      • Profesores activos: 26
      • Pares con horarios contiguos: 197
      • Pares con solapamiento de horario: 343
      • Cuentas disponibles: 26 (Base: 20, Adicionales: 6)
🔧 Definiendo el modelo de optimización...
📋 Añadiendo restricciones al modelo...
   👤 Aplicando exclusividad para 3 profesores con 3 clases
   ⏰ Aplicando restricción de no contigüidad para 197 pares
   🔄 Aplicando restricción de no solapamiento para 343 pares
✅ Modelo definido correctamente
🚀 Resolviendo el problema... (Tiempo límite: 600 segundos)

📊 Estado de la solución: Optimal

📈 RESUMEN DE CUENTAS:
   • Cuentas base utilizadas: 20/20
   • Cuentas adicionales necesarias: 4
   • TOTAL de cuentas utilizadas: 24

👥 ASIGNACIÓN DE CUENTAS POR PROFESOR:
   Profesor  NumClases  CuentaVC_Asi

# **ETAPA 3: ASIGNACIÓN DE LIBROS DIGITALES**

In [4]:
# Codigo_Etapa3_Final
import pandas as pd
import pulp
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus, PULP_CBC_CMD, value
import sys
from collections import defaultdict
import time

print("--- Iniciando Proceso de Asignación de Libros Digitales (Etapa 3 - Límite 4 por Horario) ---")
start_time = time.time()

# --- 1. Carga de Datos (Salida de la Etapa 1) ---
try:
    print("Cargando datos de asignación de clases desde la Etapa 1...")
    df_etapa1 = pd.read_excel("asignacion_profesores_E1.xlsx")
    required_cols = ['Profesor', 'Modulo', 'Horario', 'Paralelo', 'Clase_ID']
    if not all(col in df_etapa1.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df_etapa1.columns]
        raise KeyError(f"Faltan columnas esenciales: {missing}")
    df_etapa1.dropna(subset=['Modulo', 'Horario'], inplace=True)
    df_etapa1['Horario'] = df_etapa1['Horario'].astype(str).str.strip()
    if df_etapa1.empty:
        print("Error: Archivo de entrada vacío o sin datos válidos.")
        sys.exit()
    print("Datos cargados correctamente.")
except FileNotFoundError:
    print("Error: No se encontró 'asignacion_etapa1_output.xlsx'.")
    sys.exit()
except KeyError as e:
     print(f"Error de columna: {e}")
     sys.exit()
except Exception as e:
    print(f"Error al cargar o procesar archivo de Etapa 1: {e}")
    sys.exit()

# --- 2. Definición de Conjuntos y Parámetros ---
print("Definiendo conjuntos y parámetros...")

if 'Clase_ID' in df_etapa1.columns and df_etapa1['Clase_ID'].is_unique:
     print("Usando 'Clase_ID' como identificador único de clase.")
     clase_ids = df_etapa1['Clase_ID'].tolist()
     C = clase_ids
     clase_info = df_etapa1.set_index('Clase_ID').to_dict('index')
else:
     print("Advertencia: Usando índice de fila como ID de clase.")
     clases_idx = df_etapa1.index.tolist()
     C = clases_idx
     clase_info = df_etapa1.set_index(df_etapa1.index).to_dict('index')

libros = [f"Libro {i}" for i in range(1, 12)]
B = libros
modulos_presentes = df_etapa1['Modulo'].unique().tolist()
M = modulos_presentes
horarios_presentes = df_etapa1['Horario'].unique().tolist()
H = horarios_presentes

modulos_por_libro = {
    "Libro 1": {"B1", "B2"}, "Libro 2": {"B1", "B2"}, "Libro 3": {"B1", "B2"},
    "Libro 4": {"B3", "B4"}, "Libro 5": {"B3", "B4"},
    "Libro 6": {"B5", "I1"},
    "Libro 7": {"I2", "I3"},
    "Libro 8": {"I4", "A1"}, "Libro 9": {"I4", "A1"},
    "Libro 10": {"A2", "A3"},
    "Libro 11": {"A4"}
}

IsCompatible = {}
for c in C:
    # Asegurarse que clase_info[c] existe
    if c not in clase_info:
        print(f"Error crítico: ID de clase {c} no encontrado en clase_info. Revisar datos de entrada.")
        sys.exit()
    modulo_c = clase_info[c]['Modulo']
    for b in B:
        IsCompatible[b, c] = 1 if modulo_c in modulos_por_libro.get(b, set()) else 0

print(f"Clases a asignar: {len(C)}")
print(f"Licencias de libros disponibles: {len(B)}")
print(f"Horarios presentes: {H}")
print("Parámetros definidos.")

# --- 3. Definición del Modelo Matemático (PuLP) ---
print("Definiendo el modelo de optimización para libros (Límite 4 por Horario)...")

prob = LpProblem("Asignacion_Libros_Limite4Horario", LpMinimize)

x = LpVariable.dicts("AsignacionLibro", [(c, b) for c in C for b in B], cat='Binary')
u = LpVariable.dicts("UsoLibro", B, cat='Binary')

prob += lpSum(u[b] for b in B), "Minimizar_Libros_Usados"

print("Añadiendo restricciones...")

# R1: Asignación Única
for c in C:
    prob += lpSum(x[c, b] for b in B) == 1, f"AsignacionUnicaLibro_{c}"

# R2: Compatibilidad
for c in C:
    for b in B:
        prob += x[c, b] <= IsCompatible[b, c], f"Compatibilidad_{c}_{b}"

# *** RESTRICCIÓN R3 (Modificada - Límite 4 por Libro y Horario) ***
count_r3_alt = 0
limite_simultaneo = 4 # <-- LÍMITE ESTABLECIDO EN 4
print(f"Aplicando restricción R3: Límite por Libro y Horario (<= {limite_simultaneo})...")
for b in B:
    for h in H:
        # Clases en horario 'h' compatibles con libro 'b'
        clases_subset = [c for c in C if clase_info[c]['Horario'] == h and IsCompatible[b, c] == 1]
        if clases_subset:
             prob += lpSum(x[c, b] for c in clases_subset) <= limite_simultaneo, f"LimiteUsoSimultaneoHorario_{b}_{h.replace(':','').replace('-','')}"
             count_r3_alt +=1
print(f"Añadidas {count_r3_alt} restricciones de límite de uso simultáneo (<= {limite_simultaneo}).")
# *** FIN RESTRICCIÓN R3 ***

# R4: Activación de Licencia Usada
for b in B:
    for c in C:
        prob += x[c, b] <= u[b], f"ActivarUsoLibro_{c}_{b}"

# R5: Dominio (ya definido)

print("Modelo definido.")

# --- 4. Resolución del Problema ---
print("Resolviendo el problema de asignación de libros...")
solver = PULP_CBC_CMD(msg=0)
prob.solve(solver)
print("Solver finalizado.")

# --- 5. Presentación de Resultados ---
end_time = time.time()
status_text = LpStatus[prob.status]
print(f"\nTiempo total de ejecución: {end_time - start_time:.2f} segundos")
print(f"Estado de la solución final: {status_text} (Code: {prob.status})")

if prob.status == pulp.LpStatusOptimal:
    print("\n--- Resultados de la Asignación de Libros Digitales (Límite 4 por Horario) ---")

    # Calcular y mostrar resultados
    libros_usados_count = sum(u[b].varValue for b in B if hasattr(u[b], 'varValue') and u[b].varValue is not None)
    print(f"Número total de licencias de libros distintas utilizadas: {int(libros_usados_count)}")
    libros_no_usados = [b for b in B if hasattr(u[b], 'varValue') and u[b].varValue is not None and u[b].varValue < 0.1]
    if libros_no_usados:
        print(f"Licencias de libros NO utilizadas: {sorted(libros_no_usados)}")

    resultados_libros = []
    libros_asignados_detalle = defaultdict(list)
    solution_valid = True
    for c in C:
        assigned_book = "ERROR"
        for b in B:
             if hasattr(x[c, b], 'varValue') and x[c, b].varValue is not None and x[c, b].varValue > 0.9:
                 assigned_book = b
                 info = clase_info[c]
                 libros_asignados_detalle[(b, info['Horario'])].append(c)
                 break
        if assigned_book == "ERROR":
             print(f"¡Advertencia! No se asignó libro a la clase {c}")
             solution_valid = False
        info = clase_info[c]
        resultados_libros.append({
            "Clase_ID_o_Indice": c,
            "Profesor": info.get('Profesor', 'N/A'),
            "Modulo": info.get('Modulo', 'N/A'),
            "Horario": info.get('Horario', 'N/A'),
            "Paralelo": info.get('Paralelo', 'N/A'),
            "LibroDigital_Asignado": assigned_book
        })

    if not solution_valid:
         print("\nError en la asignación, no se generarán resultados detallados.")
    else:
        df_resultados_final = pd.DataFrame(resultados_libros)
        try:
            df_resultados_final.sort_values(by=["Modulo", "Horario", "Profesor"], inplace=True)
        except KeyError:
             df_resultados_final.sort_values(by=["Clase_ID_o_Indice"], inplace=True)

        print("\nAsignación Final de Libros Digitales por Clase:")
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1200):
            print(df_resultados_final)

        print(f"\nVerificación del Límite de Uso Simultáneo (<= {limite_simultaneo}):")
        violation_found = False
        max_usage_found = 0
        for key, classes in libros_asignados_detalle.items():
            libro, horario = key
            usage = len(classes)
            max_usage_found = max(max_usage_found, usage)
            if usage > limite_simultaneo:
                print(f"  ¡VIOLACIÓN ENCONTRADA! Libro: {libro}, Horario: {horario}, Usos: {usage} (> {limite_simultaneo}), Clases: {classes}")
                violation_found = True
        if not violation_found:
            print(f"  No se encontraron violaciones del límite <= {limite_simultaneo} por Libro y Horario. Uso máximo detectado: {max_usage_found}")

        # Exportación Final
        try:
            output_filename_final = "asignacion_libros_E3.xlsx"
            df_resultados_final.to_excel(output_filename_final, index=False)
            print(f"\n¡Resultados finales (Límite 4) exportados exitosamente a '{output_filename_final}'!")
        except Exception as e:
            print(f"\nError al exportar los resultados finales a Excel: {e}")

elif prob.status == pulp.LpStatusInfeasible:
    print("El problema de asignación de libros (Límite 4 por Horario) es INFACTIBLE.")
    print("Esto es inesperado con límite 4. Verifica si hay algún otro cuello de botella extremo (>4) o error en los datos.")
elif prob.status == pulp.LpStatusNotSolved:
     print("El solver no se ejecutó o fue interrumpido.")
else:
    print(f"No se encontró una solución óptima. Estado final: {status_text} (Code: {prob.status})")

print("\n--- Fin del Proceso Etapa 3 (Límite 4 por Horario) ---")

--- Iniciando Proceso de Asignación de Libros Digitales (Etapa 3 - Límite 4 por Horario) ---
Cargando datos de asignación de clases desde la Etapa 1...
Datos cargados correctamente.
Definiendo conjuntos y parámetros...
Usando 'Clase_ID' como identificador único de clase.
Clases a asignar: 46
Licencias de libros disponibles: 11
Horarios presentes: ['08H00-10H00', '17H00-19H00', '19H00-21H00', '15H00-17H00']
Parámetros definidos.
Definiendo el modelo de optimización para libros (Límite 4 por Horario)...
Añadiendo restricciones...
Aplicando restricción R3: Límite por Libro y Horario (<= 4)...
Añadidas 35 restricciones de límite de uso simultáneo (<= 4).
Modelo definido.
Resolviendo el problema de asignación de libros...
Solver finalizado.

Tiempo total de ejecución: 0.10 segundos
Estado de la solución final: Optimal (Code: 1)

--- Resultados de la Asignación de Libros Digitales (Límite 4 por Horario) ---
Número total de licencias de libros distintas utilizadas: 7
Licencias de libros NO ut